In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
disable_warnings(InsecureRequestWarning)
import re
base_url='https://kataloq.gomap.az'

In [2]:
xetali_linkler=[]

In [3]:
def get_soup(url):
    while True:
        try:
            res = requests.get(url, verify=False)
            break
        except:
            pass
    html = res.content
    soup = BeautifulSoup(html,'html.parser')
    return soup

In [4]:
def get_a_tags(url):
    
    soup=get_soup(url)
    div=soup.find('div', {"class": "flex-wrapper"})
    a_tags=div.find_all('a', {"class": 'cat-item'})
    
    return a_tags

### Kataloqlar

In [5]:
def get_catalog(url):
    kataloglar_ve_linkleri={}
    
    a_tags=get_a_tags(url)

    for row in a_tags:
        katalog_linki=base_url+row.get('href')
        katalog=row.find('div', {'class': 'x-cat-name'}).text.strip()
    
        kataloglar_ve_linkleri[katalog]=katalog_linki
        
    return kataloglar_ve_linkleri

In [6]:
cat_and_url=get_catalog('https://kataloq.gomap.az/az')
#print(cat_and_url)

### Alt kataloqlar

In [7]:
def get_sub_catalogs(url):
    
    sub_catalogs={}
    
    a_tags=get_a_tags(url)
    
    for row in a_tags:
        href=row.get('href')
        
        link=url+href[href.rfind('/'):]
        
        name=row.find('div', {"style": "float:left; align-self:center; flex-grow:1"}).text.strip()
        
        sub_catalogs[name]=link
        
    return sub_catalogs

In [8]:
def get_gurum_pages(url):
    pages=[]
    i=0
    count=2
    
    soup=get_soup(url)
    
    tapilan_netice=soup.find('div', {'class': 'x-result-found'})
    
    tapilan_netice=int(re.findall(r'\b\d+\b', tapilan_netice.text)[0])
    
    while i<=tapilan_netice//7:
        
        link=url+'?pg='+str(count)
    
        soup=get_soup(link)
    
        find_page=soup.find('ul', {'class': 'x-poilist-ul'}).text.strip()
    
        if find_page=='':
            break
            
        pages.append(link)
        
        count+=1
        i+=1
        
    pages.insert(0, url)
        
    return pages

In [9]:
def get_gurumlar(url):
    
    gurum_url_name={}
    
    gurum_urls=get_gurum_pages(url)
    
    for gurum_linki in gurum_urls:
        
        soup=get_soup(gurum_linki)
        
        ul=soup.find('ul', {'class':'x-poilist-ul'})
        
        try:
    
            a_s=ul.find_all('a')
            
            for row in a_s:
                link=row.get('href')
                name=row.text.strip()

                gurum_url_name[base_url+link]=name
                
        except:
            
            xetali_linkler.append(base_url+link)
        
    return gurum_url_name

In [10]:
def gurum_data(url):
    netice={}
    
    soup=get_soup(url)
    ul=soup.find('ul', {'class': 'x-poi-detail-list'})
    
    if not ul:
        unvan, nomre, faks, mail, web_page=['Yoxdur']*5
        
        return unvan, nomre, faks, mail, web_page
        
    else:
        
        all_li=ul.find_all('li', {'class': 'x-poi-detail-item'})

    for i in range(len(all_li)):
        cvb=all_li[i].text.strip().strip().split(':\r')
    
        netice[cvb[0]]=cvb[1].strip()
        
    if "Ünvan" in netice:
        unvan=netice['Ünvan']
        
    else:
        unvan='Yoxdur'
        
    if "Telefon" in netice:
        nomre=netice['Telefon']
        
    else:
        nomre='Yoxdur'
    
    if "Faks" in netice:
        faks=netice['Faks']
        
    else:
        faks='Yoxdur'
        
    if "E-poçt" in netice:
        mail=netice['E-poçt']
        
    else:
        mail='Yoxdur'
        
    if 'Web səhifə' in netice:
        web_page=netice['Web səhifə']
        
    else:
        web_page='Yoxdur'
        
    return (unvan, nomre, faks, mail, web_page)

In [11]:
data_file=open('data.csv', 'w', encoding='utf-8')

In [12]:
for catalog, cat_url in cat_and_url.items():
    
    print('#####',catalog, 'catalogu basladi')
    
    sub_cats=get_sub_catalogs(cat_url)
    
    for sub_cat, sub_cat_url in sub_cats.items():
        
        print(sub_cat, 'sub catalogu basladi')
        
        sirket_url_name=get_gurumlar(sub_cat_url)
        
        for sirket_url, sirket_name in sirket_url_name.items():
            
            unvan, nomre, faks, mail, web_page=gurum_data(sirket_url)
            
            #print(sirket_name, unvan, nomre, faks, mail, web_page)
            
            data_file.write(catalog+';:'+sub_cat+';:'+sirket_name+';:'+unvan+';:'+nomre+';:'+faks+';:'+mail+';:'+web_page+"\n")
            
            #break
        
        print(sub_cat, 'sub catalogu bitdi')
        #break
    
    print(catalog, 'catalogu bitdi')
    #break

##### Elm və təhsil catalogu basladi
Elmlər akademiyası sub catalogu basladi
Elmlər akademiyası sub catalogu bitdi
Kitabxanalar sub catalogu basladi
Kitabxanalar sub catalogu bitdi
Uşaq bağçaları sub catalogu basladi
Uşaq bağçaları sub catalogu bitdi
Məktəblər sub catalogu basladi
Məktəblər sub catalogu bitdi
Universitetlər sub catalogu basladi
Universitetlər sub catalogu bitdi
Kurslar sub catalogu basladi
Kurslar sub catalogu bitdi
Rəsədxana sub catalogu basladi
Rəsədxana sub catalogu bitdi
Arxivlər sub catalogu basladi
Arxivlər sub catalogu bitdi
Dəftərxana ləvazimatları sub catalogu basladi
Dəftərxana ləvazimatları sub catalogu bitdi
Kitab sub catalogu basladi
Kitab sub catalogu bitdi
Elm və təhsil catalogu bitdi
##### Mədəniyyət catalogu basladi
Otellər sub catalogu basladi
Otellər sub catalogu bitdi
Motellər sub catalogu basladi
Motellər sub catalogu bitdi
Qonaq evləri sub catalogu basladi
Qonaq evləri sub catalogu bitdi
Pansionat sub catalogu basladi
Pansionat sub catalogu bitdi


Yaradıcılıq evləri sub catalogu bitdi
Yemək çatdırma sub catalogu basladi
Yemək çatdırma sub catalogu bitdi
Kulinariya mağazaları sub catalogu basladi
Kulinariya mağazaları sub catalogu bitdi
Restoran və kafe catalogu bitdi
##### Ticarət catalogu basladi
Ərzaq sub catalogu basladi
Ərzaq sub catalogu bitdi
Geyim sub catalogu basladi
Geyim sub catalogu bitdi
Qab-qacaq sub catalogu basladi
Qab-qacaq sub catalogu bitdi
Elektrik malları sub catalogu basladi
Elektrik malları sub catalogu bitdi
Parfümeriya sub catalogu basladi
Parfümeriya sub catalogu bitdi
Dəftərxana ləvazimatları sub catalogu basladi
Dəftərxana ləvazimatları sub catalogu bitdi
Kitab sub catalogu basladi
Kitab sub catalogu bitdi
Kompüterlər sub catalogu basladi
Kompüterlər sub catalogu bitdi
Mebel sub catalogu basladi
Mebel sub catalogu bitdi
Mobil telefonlar sub catalogu basladi
Mobil telefonlar sub catalogu bitdi
Hədiyyələr sub catalogu basladi
Hədiyyələr sub catalogu bitdi
Güllər sub catalogu basladi
Güllər sub catalogu b

İdman və sağlamlıq mərkəzləri sub catalogu basladi
İdman və sağlamlıq mərkəzləri sub catalogu bitdi
Tennis kortları sub catalogu basladi
Tennis kortları sub catalogu bitdi
Fitnes-klublar sub catalogu basladi
Fitnes-klublar sub catalogu bitdi
Hovuzlar sub catalogu basladi
Hovuzlar sub catalogu bitdi
Cıdır meydanı və atçılıq sub catalogu basladi
Cıdır meydanı və atçılıq sub catalogu bitdi
Stolüstü oyunlar sub catalogu basladi
Stolüstü oyunlar sub catalogu bitdi
İdman malları sub catalogu basladi
İdman malları sub catalogu bitdi
Stadionlar sub catalogu basladi
Stadionlar sub catalogu bitdi
kinoteatr sub catalogu basladi
kinoteatr sub catalogu bitdi
teatr sub catalogu basladi
teatr sub catalogu bitdi
boulinq sub catalogu basladi
boulinq sub catalogu bitdi
bilyard sub catalogu basladi
bilyard sub catalogu bitdi
zoopark sub catalogu basladi
zoopark sub catalogu bitdi
İdman və gənclər catalogu bitdi
##### SOS catalogu basladi
Xəstəxanalar sub catalogu basladi
Xəstəxanalar sub catalogu bitdi
T